In [2]:
import pandas as pd
import numpy as np

df_annual_em_ela_2019 = pd.read_csv(
    "/Users/kashafali/Documents/Duke/Spring23/UDS/Project/Data/2019/Annual EM ELA.csv"
)


In [3]:
df_annual_em_ela_2019.columns


Index(['ENTITY_CD', 'ENTITY_NAME', 'YEAR', 'ASSESSMENT_NAME', 'SUBGROUP_NAME',
       'NUM_TESTED', 'NOT_TESTED', 'LEVEL1_COUNT', 'LEVEL1_%TESTED',
       'LEVEL2_COUNT', 'LEVEL2_%TESTED', 'LEVEL3_COUNT', 'LEVEL3_%TESTED',
       'LEVEL4_COUNT', 'LEVEL4_%TESTED', 'NUM_PROF', 'PER_PROF',
       'TOTAL_SCALE_SCORES', 'MEAN_SCORE'],
      dtype='object')

In [4]:
df_annual_em_ela_2019["SUBGROUP_NAME"].unique()

array(['Hispanic or Latino', 'Homeless', 'Male',
       'Non-English Language Learners', 'Not Economically Disadvantaged',
       'American Indian or Alaska Native',
       'Asian or Native Hawaiian/Other Pacific Islander',
       'Black or African American', 'Economically Disadvantaged',
       'English Language Learners', 'Female', 'General Education',
       'Multiracial', 'All Students', 'In Foster Care', 'Migrant',
       'Not Homeless', 'Not in Foster Care', 'Not Migrant',
       'Parent in Armed Forces', 'Parent Not in Armed Forces',
       'Students with Disabilities', 'White', 'Small Group Total'],
      dtype=object)

In [5]:
# rename Non-English Language Learners to Non-English Language Learner
df_annual_em_ela_2019["SUBGROUP_NAME"] = df_annual_em_ela_2019["SUBGROUP_NAME"].replace("Non-English Language Learners", "Non-English Language Learner")
# rename English Language Learners to English Language Learner
df_annual_em_ela_2019["SUBGROUP_NAME"] = df_annual_em_ela_2019["SUBGROUP_NAME"].replace("English Language Learners", "English Language Learner")


# Dataframe for All Students

In [6]:
# only keep columns
column_names = [
    "ENTITY_CD",
    "ENTITY_NAME",
    "YEAR",
    "ASSESSMENT_NAME",
    "SUBGROUP_NAME",
    "NOT_TESTED",
    "NUM_TESTED",
    "NUM_PROF",
    "PER_PROF",
    "TOTAL_SCALE_SCORES",
    "MEAN_SCORE",
]

df_annual_em_ela_2019_total = df_annual_em_ela_2019[column_names]
df_annual_em_ela_2019_total = df_annual_em_ela_2019_total[
    df_annual_em_ela_2019_total["SUBGROUP_NAME"] == "All Students"
]


In [7]:
df_annual_em_ela_2019_total.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,SUBGROUP_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE
41,1,NYC Public Schools,2018,ELA5,All Students,8982,68439,25990,38,41110322,601
42,1,NYC Public Schools,2018,ELA3_8,All Students,51959,396804,185099,47,00,00
43,1,NYC Public Schools,2018,ELA3,All Students,10410,67237,34036,51,40361558,600
44,1,NYC Public Schools,2018,ELA7,All Students,7157,65250,27812,43,39254856,602
45,1,NYC Public Schools,2018,ELA8,All Students,8457,63183,32042,51,38019693,602


In [8]:
df_annual_em_ela_2019_merge = df_annual_em_ela_2019_total.drop(
    columns=["SUBGROUP_NAME"]
).copy()


# Dataframe for Gender

In [9]:
df_annual_em_ela_2019_gender = df_annual_em_ela_2019.copy()

df_annual_em_ela_2019_gender = df_annual_em_ela_2019_gender[column_names]

df_annual_em_ela_2019_gender = df_annual_em_ela_2019_gender[
    (df_annual_em_ela_2019_gender["SUBGROUP_NAME"] == "Female")
    | (df_annual_em_ela_2019_gender["SUBGROUP_NAME"] == "Male")
]


In [10]:
# fix index
df_annual_em_ela_2019_gender = df_annual_em_ela_2019_gender.set_index(
    ["SUBGROUP_NAME", "ENTITY_CD", "YEAR", "ASSESSMENT_NAME"]
).unstack(["SUBGROUP_NAME"])

df_annual_em_ela_2019_gender = df_annual_em_ela_2019_gender.reindex(
    columns=sorted(df_annual_em_ela_2019_gender.columns, key=lambda x: x[::-1])
)

df_annual_em_ela_2019_gender.columns = [
    "{}_{}".format(t, v) for v, t in df_annual_em_ela_2019_gender.columns
]

df_annual_em_ela_2019_gender = pd.DataFrame(df_annual_em_ela_2019_gender.to_records())


In [11]:
df_annual_em_ela_2019_gender.head()


,ENTITY_CD,YEAR,ASSESSMENT_NAME,Female_ENTITY_NAME,Female_MEAN_SCORE,Female_NOT_TESTED,Female_NUM_PROF,Female_NUM_TESTED,Female_PER_PROF,Female_TOTAL_SCALE_SCORES,Male_ENTITY_NAME,Male_MEAN_SCORE,Male_NOT_TESTED,Male_NUM_PROF,Male_NUM_TESTED,Male_PER_PROF,Male_TOTAL_SCALE_SCORES
0,1,2018,ELA3,NYC Public Schools,603,4030.0,18392,32870.0,56,19821112,NYC Public Schools,598,6380.0,15644,34367.0,46,20540446
1,1,2018,ELA4,NYC Public Schools,604,3822.0,18016,33094.0,54,19982478,NYC Public Schools,598,5635.0,15340,34493.0,44,20643286
2,1,2018,ELA5,NYC Public Schools,604,3532.0,14523,33688.0,43,20340275,NYC Public Schools,598,5450.0,11467,34751.0,33,20770047
3,1,2018,ELA6,NYC Public Schools,604,3044.0,18035,31926.0,56,19292313,NYC Public Schools,597,4452.0,13828,33182.0,42,19797315
4,1,2018,ELA7,NYC Public Schools,605,2877.0,15652,31599.0,50,19126216,NYC Public Schools,598,4280.0,12160,33651.0,36,20128640


# Merge Gender Dataframe with All Students Dataframe

In [12]:
df_ela_2019_1 = pd.merge(
    df_annual_em_ela_2019_merge,
    df_annual_em_ela_2019_gender,
    on=["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],
    how="left",
)

df_ela_2019_1.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,Female_NUM_TESTED,Female_PER_PROF,Female_TOTAL_SCALE_SCORES,Male_ENTITY_NAME,Male_MEAN_SCORE,Male_NOT_TESTED,Male_NUM_PROF,Male_NUM_TESTED,Male_PER_PROF,Male_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,ELA5,8982,68439,25990,38,41110322,601,...,33688.0,43,20340275,NYC Public Schools,598,5450.0,11467,34751.0,33,20770047
1,1,NYC Public Schools,2018,ELA3_8,51959,396804,185099,47,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NYC Public Schools,2018,ELA3,10410,67237,34036,51,40361558,600,...,32870.0,56,19821112,NYC Public Schools,598,6380.0,15644,34367.0,46,20540446
3,1,NYC Public Schools,2018,ELA7,7157,65250,27812,43,39254856,602,...,31599.0,50,19126216,NYC Public Schools,598,4280.0,12160,33651.0,36,20128640
4,1,NYC Public Schools,2018,ELA8,8457,63183,32042,51,38019693,602,...,30665.0,58,18565377,NYC Public Schools,598,4815.0,14291,32518.0,44,19454316


# Dataframe for Race

In [13]:
df_annual_em_ela_2019_race = df_annual_em_ela_2019[column_names].copy()

df_annual_em_ela_2019_race = df_annual_em_ela_2019_race[
    (df_annual_em_ela_2019_race["SUBGROUP_NAME"] == "American Indian or Alaska Native")
    | (df_annual_em_ela_2019_race["SUBGROUP_NAME"] == "Black or African American")
    | (df_annual_em_ela_2019_race["SUBGROUP_NAME"] == "Hispanic or Latino")
    | (
        df_annual_em_ela_2019_race["SUBGROUP_NAME"]
        == "Asian or Native Hawaiian/Other Pacific Islander"
    )
    | (df_annual_em_ela_2019_race["SUBGROUP_NAME"] == "White")
    | (df_annual_em_ela_2019_race["SUBGROUP_NAME"] == "Multiracial")
]


In [14]:
# reindex

df_annual_em_ela_2019_race = df_annual_em_ela_2019_race.set_index(
    ["SUBGROUP_NAME", "ENTITY_CD", "YEAR", "ASSESSMENT_NAME"]
).unstack(["SUBGROUP_NAME"])

df_annual_em_ela_2019_race = df_annual_em_ela_2019_race.reindex(
    columns=sorted(df_annual_em_ela_2019_race.columns, key=lambda x: x[::-1])
)

df_annual_em_ela_2019_race.columns = [
    "{}_{}".format(t, v) for v, t in df_annual_em_ela_2019_race.columns
]

df_annual_em_ela_2019_race = pd.DataFrame(df_annual_em_ela_2019_race.to_records())


In [15]:
df_annual_em_ela_2019_race.head()


,ENTITY_CD,YEAR,ASSESSMENT_NAME,American Indian or Alaska Native_ENTITY_NAME,American Indian or Alaska Native_MEAN_SCORE,American Indian or Alaska Native_NOT_TESTED,American Indian or Alaska Native_NUM_PROF,American Indian or Alaska Native_NUM_TESTED,American Indian or Alaska Native_PER_PROF,American Indian or Alaska Native_TOTAL_SCALE_SCORES,...,Multiracial_NUM_TESTED,Multiracial_PER_PROF,Multiracial_TOTAL_SCALE_SCORES,White_ENTITY_NAME,White_MEAN_SCORE,White_NOT_TESTED,White_NUM_PROF,White_NUM_TESTED,White_PER_PROF,White_TOTAL_SCALE_SCORES
0,1,2018,ELA3,NYC Public Schools,599,96.0,348,756.0,46,452614,...,937.0,72,572429,NYC Public Schools,609,3433.0,7869,11416.0,69,6950213
1,1,2018,ELA4,NYC Public Schools,599,83.0,326,726.0,45,435113,...,817.0,76,501907,NYC Public Schools,611,3286.0,7638,11000.0,69,6720403
2,1,2018,ELA5,NYC Public Schools,599,91.0,256,762.0,34,456387,...,807.0,63,494431,NYC Public Schools,610,3189.0,6383,11063.0,58,6745329
3,1,2018,ELA6,NYC Public Schools,598,95.0,322,696.0,46,416506,...,641.0,76,392825,NYC Public Schools,610,2601.0,7357,10467.0,70,6386851
4,1,2018,ELA7,NYC Public Schools,599,76.0,289,738.0,39,442109,...,625.0,71,383921,NYC Public Schools,611,2426.0,6680,10365.0,64,6332714


# Merge Race Dataset with All Students + Gender Dataset

In [16]:
df_ela_2019_2 = pd.merge(
    df_ela_2019_1,
    df_annual_em_ela_2019_race,
    on=["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],
    how="left",
)

df_ela_2019_2.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,Multiracial_NUM_TESTED,Multiracial_PER_PROF,Multiracial_TOTAL_SCALE_SCORES,White_ENTITY_NAME,White_MEAN_SCORE,White_NOT_TESTED,White_NUM_PROF,White_NUM_TESTED,White_PER_PROF,White_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,ELA5,8982,68439,25990,38,41110322,601,...,807.0,63,494431,NYC Public Schools,610,3189.0,6383,11063.0,58,6745329
1,1,NYC Public Schools,2018,ELA3_8,51959,396804,185099,47,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NYC Public Schools,2018,ELA3,10410,67237,34036,51,40361558,600,...,937.0,72,572429,NYC Public Schools,609,3433.0,7869,11416.0,69,6950213
3,1,NYC Public Schools,2018,ELA7,7157,65250,27812,43,39254856,602,...,625.0,71,383921,NYC Public Schools,611,2426.0,6680,10365.0,64,6332714
4,1,NYC Public Schools,2018,ELA8,8457,63183,32042,51,38019693,602,...,522.0,73,319625,NYC Public Schools,609,2800.0,6707,9850.0,68,6002917


# English Language Learners Dataframe

In [17]:
df_annual_em_ela_2019_eng = df_annual_em_ela_2019[column_names].copy()


df_annual_em_ela_2019_eng = df_annual_em_ela_2019_eng[
    (df_annual_em_ela_2019_eng["SUBGROUP_NAME"] == "Non-English Language Learner")
    | (df_annual_em_ela_2019_eng["SUBGROUP_NAME"] == "English Language Learner")
]


In [18]:
# reindex

df_annual_em_ela_2019_eng = df_annual_em_ela_2019_eng.set_index(
    ["SUBGROUP_NAME", "ENTITY_CD", "YEAR", "ASSESSMENT_NAME"]
).unstack(["SUBGROUP_NAME"])

df_annual_em_ela_2019_eng = df_annual_em_ela_2019_eng.reindex(
    columns=sorted(df_annual_em_ela_2019_eng.columns, key=lambda x: x[::-1])
)

df_annual_em_ela_2019_eng.columns = [
    "{}_{}".format(t, v) for v, t in df_annual_em_ela_2019_eng.columns
]

df_annual_em_ela_2019_eng = pd.DataFrame(df_annual_em_ela_2019_eng.to_records())


In [19]:
df_annual_em_ela_2019_eng.head()


,ENTITY_CD,YEAR,ASSESSMENT_NAME,English Language Learner_ENTITY_NAME,English Language Learner_MEAN_SCORE,English Language Learner_NOT_TESTED,English Language Learner_NUM_PROF,English Language Learner_NUM_TESTED,English Language Learner_PER_PROF,English Language Learner_TOTAL_SCALE_SCORES,Non-English Language Learner_ENTITY_NAME,Non-English Language Learner_MEAN_SCORE,Non-English Language Learner_NOT_TESTED,Non-English Language Learner_NUM_PROF,Non-English Language Learner_NUM_TESTED,Non-English Language Learner_PER_PROF,Non-English Language Learner_TOTAL_SCALE_SCORES
0,1,2018,ELA3,NYC Public Schools,586,2604.0,2412,11825.0,20,6932450,NYC Public Schools,603,7806.0,31624,55412.0,57,33429108
1,1,2018,ELA4,NYC Public Schools,584,2391.0,1286,9842.0,13,5744189,NYC Public Schools,604,7066.0,32070,57745.0,56,34881575
2,1,2018,ELA5,NYC Public Schools,580,2120.0,262,8330.0,03,4828302,NYC Public Schools,604,6862.0,25728,60109.0,43,36282020
3,1,2018,ELA6,NYC Public Schools,579,2002.0,510,7709.0,07,4462824,NYC Public Schools,603,5494.0,31353,57399.0,55,34626804
4,1,2018,ELA7,NYC Public Schools,579,1953.0,203,7019.0,03,4061768,NYC Public Schools,604,5204.0,27609,58231.0,47,35193088


# Merge English Language Learners Dataset with All Students + Gender + Race Dataset

In [20]:
df_ela_2019_3 = pd.merge(
    df_ela_2019_2,
    df_annual_em_ela_2019_eng,
    on=["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],
    how="left",
)

df_ela_2019_3.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,English Language Learner_NUM_TESTED,English Language Learner_PER_PROF,English Language Learner_TOTAL_SCALE_SCORES,Non-English Language Learner_ENTITY_NAME,Non-English Language Learner_MEAN_SCORE,Non-English Language Learner_NOT_TESTED,Non-English Language Learner_NUM_PROF,Non-English Language Learner_NUM_TESTED,Non-English Language Learner_PER_PROF,Non-English Language Learner_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,ELA5,8982,68439,25990,38,41110322,601,...,8330.0,03,4828302,NYC Public Schools,604,6862.0,25728,60109.0,43,36282020
1,1,NYC Public Schools,2018,ELA3_8,51959,396804,185099,47,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NYC Public Schools,2018,ELA3,10410,67237,34036,51,40361558,600,...,11825.0,20,6932450,NYC Public Schools,603,7806.0,31624,55412.0,57,33429108
3,1,NYC Public Schools,2018,ELA7,7157,65250,27812,43,39254856,602,...,7019.0,03,4061768,NYC Public Schools,604,5204.0,27609,58231.0,47,35193088
4,1,NYC Public Schools,2018,ELA8,8457,63183,32042,51,38019693,602,...,6704.0,05,3872714,NYC Public Schools,605,6335.0,31727,56479.0,56,34146979


# Dataframe for Economic Condition

In [21]:
df_annual_em_ela_2019_econ = df_annual_em_ela_2019[column_names].copy()

df_annual_em_ela_2019_econ = df_annual_em_ela_2019_econ[
    (df_annual_em_ela_2019_econ["SUBGROUP_NAME"] == "Economically Disadvantaged")
    | (df_annual_em_ela_2019_econ["SUBGROUP_NAME"] == "Not Economically Disadvantaged")
]


In [22]:
# reindex

df_annual_em_ela_2019_econ = df_annual_em_ela_2019_econ.set_index(
    ["SUBGROUP_NAME", "ENTITY_CD", "YEAR", "ASSESSMENT_NAME"]
).unstack(["SUBGROUP_NAME"])

df_annual_em_ela_2019_econ = df_annual_em_ela_2019_econ.reindex(
    columns=sorted(df_annual_em_ela_2019_econ.columns, key=lambda x: x[::-1])
)

df_annual_em_ela_2019_econ.columns = [
    "{}_{}".format(t, v) for v, t in df_annual_em_ela_2019_econ.columns
]

df_annual_em_ela_2019_econ = pd.DataFrame(df_annual_em_ela_2019_econ.to_records())


In [23]:
df_annual_em_ela_2019_econ.head()


,ENTITY_CD,YEAR,ASSESSMENT_NAME,Economically Disadvantaged_ENTITY_NAME,Economically Disadvantaged_MEAN_SCORE,Economically Disadvantaged_NOT_TESTED,Economically Disadvantaged_NUM_PROF,Economically Disadvantaged_NUM_TESTED,Economically Disadvantaged_PER_PROF,Economically Disadvantaged_TOTAL_SCALE_SCORES,Not Economically Disadvantaged_ENTITY_NAME,Not Economically Disadvantaged_MEAN_SCORE,Not Economically Disadvantaged_NOT_TESTED,Not Economically Disadvantaged_NUM_PROF,Not Economically Disadvantaged_NUM_TESTED,Not Economically Disadvantaged_PER_PROF,Not Economically Disadvantaged_TOTAL_SCALE_SCORES
0,1,2018,ELA3,NYC Public Schools,597,5527.0,21915,50579.0,43,30182045,NYC Public Schools,611,4883.0,12121,16658.0,73,10179513
1,1,2018,ELA4,NYC Public Schools,598,4837.0,21614,51193.0,42,30589790,NYC Public Schools,612,4620.0,11742,16394.0,72,10035974
2,1,2018,ELA5,NYC Public Schools,597,4376.0,15905,51391.0,31,30700660,NYC Public Schools,611,4606.0,10085,17048.0,59,10409662
3,1,2018,ELA6,NYC Public Schools,597,3820.0,21322,50121.0,43,29936580,NYC Public Schools,611,3676.0,10541,14987.0,70,9153048
4,1,2018,ELA7,NYC Public Schools,599,3604.0,17918,49572.0,36,29680655,NYC Public Schools,611,3553.0,9894,15678.0,63,9574201


# Merge Economic Dataset with All Students + Gender + Race + English Language Dataset

In [24]:
df_ela_2019_4 = pd.merge(
    df_ela_2019_3,
    df_annual_em_ela_2019_econ,
    on=["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],
    how="left",
)

df_ela_2019_4.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,Economically Disadvantaged_NUM_TESTED,Economically Disadvantaged_PER_PROF,Economically Disadvantaged_TOTAL_SCALE_SCORES,Not Economically Disadvantaged_ENTITY_NAME,Not Economically Disadvantaged_MEAN_SCORE,Not Economically Disadvantaged_NOT_TESTED,Not Economically Disadvantaged_NUM_PROF,Not Economically Disadvantaged_NUM_TESTED,Not Economically Disadvantaged_PER_PROF,Not Economically Disadvantaged_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,ELA5,8982,68439,25990,38,41110322,601,...,51391.0,31,30700660,NYC Public Schools,611,4606.0,10085,17048.0,59,10409662
1,1,NYC Public Schools,2018,ELA3_8,51959,396804,185099,47,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NYC Public Schools,2018,ELA3,10410,67237,34036,51,40361558,600,...,50579.0,43,30182045,NYC Public Schools,611,4883.0,12121,16658.0,73,10179513
3,1,NYC Public Schools,2018,ELA7,7157,65250,27812,43,39254856,602,...,49572.0,36,29680655,NYC Public Schools,611,3553.0,9894,15678.0,63,9574201
4,1,NYC Public Schools,2018,ELA8,8457,63183,32042,51,38019693,602,...,47734.0,45,28601323,NYC Public Schools,610,4044.0,10447,15449.0,68,9418370


# Dataframe for Special Education

In [25]:
df_annual_em_ela_2019_dis = df_annual_em_ela_2019[column_names].copy()

df_annual_em_ela_2019_dis = df_annual_em_ela_2019_dis[
    (df_annual_em_ela_2019_dis["SUBGROUP_NAME"] == "Students with Disabilities")
]


In [26]:
# reindex

df_annual_em_ela_2019_dis = df_annual_em_ela_2019_dis.set_index(
    ["SUBGROUP_NAME", "ENTITY_CD", "YEAR", "ASSESSMENT_NAME"]
).unstack(["SUBGROUP_NAME"])

df_annual_em_ela_2019_dis = df_annual_em_ela_2019_dis.reindex(
    columns=sorted(df_annual_em_ela_2019_dis.columns, key=lambda x: x[::-1])
)

df_annual_em_ela_2019_dis.columns = [
    "{}_{}".format(t, v) for v, t in df_annual_em_ela_2019_dis.columns
]

df_annual_em_ela_2019_dis = pd.DataFrame(df_annual_em_ela_2019_dis.to_records())


In [27]:
df_annual_em_ela_2019_dis.head()


,ENTITY_CD,YEAR,ASSESSMENT_NAME,Students with Disabilities_ENTITY_NAME,Students with Disabilities_MEAN_SCORE,Students with Disabilities_NOT_TESTED,Students with Disabilities_NUM_PROF,Students with Disabilities_NUM_TESTED,Students with Disabilities_PER_PROF,Students with Disabilities_TOTAL_SCALE_SCORES
0,1,2018,ELA3,NYC Public Schools,585,7291,3034,14366,21,8402968
1,1,2018,ELA4,NYC Public Schools,585,6656,2753,14934,18,8737959
2,1,2018,ELA5,NYC Public Schools,585,6375,1648,15138,11,8851450
3,1,2018,ELA6,NYC Public Schools,584,5192,2262,14380,16,8401167
4,1,2018,ELA7,NYC Public Schools,586,4963,1695,14230,12,8343036


# Merge Special Education Dataset with All Students + Gender + Race + English Language + Econ Dataset

In [28]:
df_ela_2019_5 = pd.merge(
    df_ela_2019_4,
    df_annual_em_ela_2019_dis,
    on=["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],
    how="left",
)

df_ela_2019_5.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,Not Economically Disadvantaged_NUM_TESTED,Not Economically Disadvantaged_PER_PROF,Not Economically Disadvantaged_TOTAL_SCALE_SCORES,Students with Disabilities_ENTITY_NAME,Students with Disabilities_MEAN_SCORE,Students with Disabilities_NOT_TESTED,Students with Disabilities_NUM_PROF,Students with Disabilities_NUM_TESTED,Students with Disabilities_PER_PROF,Students with Disabilities_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,ELA5,8982,68439,25990,38,41110322,601,...,17048.0,59,10409662,NYC Public Schools,585,6375.0,1648,15138.0,11,8851450
1,1,NYC Public Schools,2018,ELA3_8,51959,396804,185099,47,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NYC Public Schools,2018,ELA3,10410,67237,34036,51,40361558,600,...,16658.0,73,10179513,NYC Public Schools,585,7291.0,3034,14366.0,21,8402968
3,1,NYC Public Schools,2018,ELA7,7157,65250,27812,43,39254856,602,...,15678.0,63,9574201,NYC Public Schools,586,4963.0,1695,14230.0,12,8343036
4,1,NYC Public Schools,2018,ELA8,8457,63183,32042,51,38019693,602,...,15449.0,68,9418370,NYC Public Schools,586,4995.0,2187,13185.0,17,7731223


# Clean final ELA dataset

In [29]:
# show all columns in dataframe

# Drop Entity Name columns

df_ela_2019_5 = df_ela_2019_5.drop(
    columns=['Female_ENTITY_NAME', 
    'Male_ENTITY_NAME',
    'American Indian or Alaska Native_ENTITY_NAME',
    'Black or African American_ENTITY_NAME',
    'Hispanic or Latino_ENTITY_NAME',
    'Asian or Native Hawaiian/Other Pacific Islander_ENTITY_NAME',
    'White_ENTITY_NAME',
    'Multiracial_ENTITY_NAME',
    'Non-English Language Learner_ENTITY_NAME',
    'English Language Learner_ENTITY_NAME',
    'Economically Disadvantaged_ENTITY_NAME',
    'Not Economically Disadvantaged_ENTITY_NAME',
    'Students with Disabilities_ENTITY_NAME']
)



In [30]:
df_ela_2019_5.head()

,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,Not Economically Disadvantaged_NUM_PROF,Not Economically Disadvantaged_NUM_TESTED,Not Economically Disadvantaged_PER_PROF,Not Economically Disadvantaged_TOTAL_SCALE_SCORES,Students with Disabilities_MEAN_SCORE,Students with Disabilities_NOT_TESTED,Students with Disabilities_NUM_PROF,Students with Disabilities_NUM_TESTED,Students with Disabilities_PER_PROF,Students with Disabilities_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,ELA5,8982,68439,25990,38,41110322,601,...,10085,17048.0,59,10409662,585,6375.0,1648,15138.0,11,8851450
1,1,NYC Public Schools,2018,ELA3_8,51959,396804,185099,47,00,00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NYC Public Schools,2018,ELA3,10410,67237,34036,51,40361558,600,...,12121,16658.0,73,10179513,585,7291.0,3034,14366.0,21,8402968
3,1,NYC Public Schools,2018,ELA7,7157,65250,27812,43,39254856,602,...,9894,15678.0,63,9574201,586,4963.0,1695,14230.0,12,8343036
4,1,NYC Public Schools,2018,ELA8,8457,63183,32042,51,38019693,602,...,10447,15449.0,68,9418370,586,4995.0,2187,13185.0,17,7731223


In [31]:
# save to csv

df_ela_2019_5.to_csv('df_ela_2019.csv', index=False)